In [31]:
import numpy as np
import pandas as pd
import trajectory_planning_helpers as tph
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.collections import LineCollection
from f1tenth_benchmarks.utils.MapData import MapData
from f1tenth_benchmarks.utils.track_utils import RaceTrack, CentreLine
from trajectory_planning_helpers.calc_head_curv_num import calc_head_curv_num
from scipy import interpolate


In [32]:

def count_corners(kappa, lower_threshold=0.2, corner_threshold=0.6):
    kappa = np.abs(kappa)
    corner_count = 0
    valid_crossing = True
    corner_crossings = []
    for i in range(len(kappa) -1):
        if kappa[i] > corner_threshold and valid_crossing:
            corner_count += 1
            corner_crossings.append(i)
            valid_crossing = False
        if kappa[i] < lower_threshold:
            valid_crossing = True

    return corner_count, corner_crossings

def measure_straight_section(kappa, el_lengths_full, threshold=0.1):
    kappa = np.abs(kappa)
    count = np.where(kappa < threshold)[0]
    straight_length = np.sum(el_lengths_full[count])

    return straight_length

In [33]:
map_file_list = glob.glob(f"../../maps/*_centerline.csv")

map_data = []
for map_file in map_file_list:
    track = np.loadtxt(map_file, delimiter=',', skiprows=1)

    map_name = map_file.split("/")[-1].split("_")[0]
    el_lengths = np.linalg.norm(np.diff(track[:, :2], axis=0), axis=1)
    track_length = np.sum(el_lengths)

    psi, kappa = tph.calc_head_curv_num.calc_head_curv_num(track[:, :2], el_lengths, False)

    last_el = np.linalg.norm(track[0, :2] - track[-1, :2])
    full_el = np.append(el_lengths, last_el)
    straight_length = measure_straight_section(kappa, full_el)
    corner_count, corner_crossings = count_corners(kappa)


    mini_dict = {
        "MapName": map_name, 
        "TrackLength": track_length, 
        "StraightLength": straight_length/track_length*100,
        "CornerCount": corner_count,
        }
    map_data.append(mini_dict)

df = pd.DataFrame(map_data)
df = df.sort_values(by=["MapName"])
neat_names = ["Map name", "Track length [m]", "Straight length [\%]", "Number of corners"]
df.rename(columns=dict(zip(df.columns, neat_names)), inplace=True)
df = df.set_index('Map name', drop=True).T
df.rename(columns=str.upper, inplace=True)
#df.to_latex("../Data/MapInfo/MapTable.tex", float_format="%.2f")

print(df)



Map name                    AUT         ESP         GBR         MCO
Track length [m]      94.901903  236.928941  201.837814  178.708344
Straight length [\%]  64.919514   58.981774   59.452985   60.597009
Number of corners      7.000000    7.000000    7.000000   16.000000


In [34]:
map_file_list = glob.glob(f"../../maps/*_centerline.csv")

map_data = []
for map_file in map_file_list:
    track = np.loadtxt(map_file, delimiter=',', skiprows=1)

    map_name = map_file.split("/")[-1].split("_")[0]
    el_lengths = np.linalg.norm(np.diff(track[:, :2], axis=0), axis=1)
    track_length = np.sum(el_lengths)

    psi, kappa = tph.calc_head_curv_num.calc_head_curv_num(track[:, :2], el_lengths, False)

    total_curvature = np.sum(np.abs(kappa)) # consider integrating over distance...?
    max_curvature = np.max(np.abs(kappa))
    mean_curvature = np.mean(np.abs(kappa))

    last_el = np.linalg.norm(track[0, :2] - track[-1, :2])
    full_el = np.append(el_lengths, last_el)
    straight_length = measure_straight_section(kappa, full_el)
    corner_count, corner_crossings = count_corners(kappa)

    corrected = kappa * full_el
    total_corrected = np.sum(np.abs(corrected))

    mini_dict = {
        "MapName": map_name, 
        "TrackLength": track_length, 
        "TotalCurvature": total_curvature, 
        # "TotalCorrected": total_curvature * track_length,
        "TotalCorrected": total_corrected,
        "MeanCurvature": mean_curvature,
    #  "MaxCurvature": max_curvature, 
        "StraightLength": straight_length/track_length*100,
        "CornerCount": corner_count,
        }
    map_data.append(mini_dict)

df = pd.DataFrame(map_data)
df.CornerCount = df.CornerCount.astype(int)
df = df.sort_values(by=["MapName"])
neat_names = ["Map name", "Track length [m]", "Total curvature", "Total corrected [rad]", "Mean curvature", "Straight length [m]", "Corner count"]
# df.rename(columns=neat_names, inplace=True)
df.rename(columns=dict(zip(df.columns, neat_names)), inplace=True)
df = df.set_index('Map name', drop=True)
# df.set_axis(neat_names, axis=1, inplace=True)
df = df.T
df.rename(columns=str.upper, inplace=True)
#df.to_latex("../Data/MapInfo/MapTable.tex", float_format="%.2f")

print(df)

map_df = df.T
map_df = map_df.reset_index()
print(map_df)

Map name                     AUT         ESP         GBR         MCO
Track length [m]       94.901903  236.928941  201.837814  178.708344
Total curvature        82.632845  158.752590  156.193379  171.391243
Total corrected [rad]  16.562513   31.948041   31.538246   34.358199
Mean curvature          0.174331    0.134308    0.155108    0.192143
Straight length [m]    64.919514   58.981774   59.452985   60.597009
Corner count            7.000000    7.000000    7.000000   16.000000
  Map name  Track length [m]  Total curvature  Total corrected [rad]  \
0      AUT         94.901903        82.632845              16.562513   
1      ESP        236.928941       158.752590              31.948041   
2      GBR        201.837814       156.193379              31.538246   
3      MCO        178.708344       171.391243              34.358199   

   Mean curvature  Straight length [m]  Corner count  
0        0.174331            64.919514           7.0  
1        0.134308            58.981774        

In [35]:
def numerical_sort(value):
    numbers = re.findall(r'\d+', value)
    return int(numbers[-1]) if numbers else float('inf')

In [40]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, Layout
from scipy.signal import correlate, find_peaks

def interpolate_track_new(points, n_points=None, s=0):
    if len(points) <= 1:
        return points
    order_k = min(3, len(points) - 1)
    tck = interpolate.splprep([points[:, 0], points[:, 1]], k=order_k, s=s)[0]
    if n_points is None: n_points = len(points)
    track = np.array(interpolate.splev(np.linspace(0, 1, n_points), tck)).T
    return track

def resample_track_points(points, seperation_distance=0.2, smoothing=0.2):
    if points[0, 0] > points[-1, 0]:
        points = np.flip(points, axis=0)

    line_length = np.sum(np.linalg.norm(np.diff(points, axis=0), axis=1))
    n_pts = max(int(line_length / seperation_distance), 2)
    smooth_line = interpolate_track_new(points, None, smoothing)
    resampled_points = interpolate_track_new(smooth_line, n_pts, 0)

    return resampled_points, smooth_line

def plot_with_slider(n):
    # Load the boundary files based on the slider value for `n`
    local_track = np.load("../../Logs/LocalMPCC/RawData_mu60/LocalMapData_mu60/local_map_"+ str(n) +".npy")
    print(f"Local_track size: {local_track.shape}, length: {len(local_track)}")
    map_names = map_df["Map name"].values

    el_lengthsLocal = np.sqrt(np.sum(np.diff(local_track, axis=0)**2, axis=1))
    psiLocal, kappaLocal = calc_head_curv_num(
            path=local_track,
            el_lengths=el_lengthsLocal,
            is_closed=False,
        )
    map_names = ["gbr"]
    for i, map_name in enumerate(map_names):
        centre_line = CentreLine(map_name.lower(), directory="../../maps/")  
        track = centre_line
        kappa = centre_line.kappa
        resampled_points, smooth_line = resample_track_points(centre_line.path, seperation_distance=0.2, smoothing=0.5)
        number_of_track_points = len(kappa)
        print(f"Number of track points: {number_of_track_points}")
        corner_count, corner_crossings = count_corners(kappa)
        print(f"Corner count: {corner_count}")

        last_el = np.linalg.norm(centre_line.path[0] - centre_line.path[-1])
        full_el = np.append(centre_line.el_lengths, last_el)
        straight_length = measure_straight_section(kappa, full_el)
        print(f"Straight length: {straight_length} m")
        
        el_lengthsSmooth = np.sqrt(np.sum(np.diff(smooth_line, axis=0)**2, axis=1))
        psiSmooth, kappaSmooth = calc_head_curv_num(
                path=smooth_line,
                el_lengths=el_lengthsSmooth,
                is_closed=False,
            )

        el_lengthsSmooth2 = np.sqrt(np.sum(np.diff(resampled_points, axis=0)**2, axis=1))
        print(f'Average lenght between points on resampled map: {np.mean(el_lengthsSmooth2)}')
        print(f'Average lenght between points on local map: {np.mean(el_lengthsLocal)}')
        psiSmooth2, kappaSmooth2 = calc_head_curv_num(
                path=resampled_points,
                el_lengths=el_lengthsSmooth2,
                is_closed=False,
            )
        
        start_index = n
        
        correlation_scores = np.correlate(np.flip(-kappaSmooth2), kappaLocal, mode='same')
        print(f"Corrleation scores lenght:{len(correlation_scores)}")
        min_score = np.min(correlation_scores)
        shifted_scores = correlation_scores - min_score # Shift the scores to ensure all are positive (if necessary) >=0
        normalized_scores = shifted_scores / np.max(shifted_scores) # Normalize the scores to a range between 0 and 1
        probabilities = normalized_scores / np.sum(normalized_scores) # Convert to probabilities by dividing by the sum of the normalized scores
        print(f"Probabilities lenght: {len(probabilities)}")
        ma_sum = np.sum(probabilities) #Ensure probabilities sumise to 1
        print(f"Sum of probabilities: {ma_sum}")
        
        # Replace negative values with zero
        non_negative_scores = np.where(correlation_scores < 0, 0, correlation_scores)
        probabilities = non_negative_scores / np.sum(non_negative_scores)
       
        
        peaks, properties = find_peaks(correlation_scores, prominence=0.2)  # Adjust prominence as needed
        estimated_position_peak = np.argmax(correlation_scores)       # Find the peak closest to the actual segment position (for visualization purposes)
            
        fig, axs = plt.subplots(3, 1, figsize=(10, 8))
     
        end_index = start_index + len(kappaLocal)
        axs[0].plot(kappa, label='Raw track Curvature') # This should be flipped
        axs[0].plot(np.flip(-kappaSmooth2), label='Resampled track Curvature')
        axs[0].plot(np.arange(start_index, end_index), kappaLocal, linewidth=2, label='Segment (Local Curvature)', color='red')
        axs[0].set_xlim(0, number_of_track_points)
        axs[0].set_title(f"Curvature of '{map_name}' track")
        
        shift_x = np.linspace(0, len(correlation_scores), len(correlation_scores))
        axs[1].plot(shift_x, probabilities, label="Correlation Scores", color='green')
        axs[1].axvline(n, color='blue', linestyle='--', label='True Peak Position', linewidth=2)
        axs[1].plot(peaks, probabilities[peaks], "x", label="Detected Peaks", color='red')
        axs[1].set_title("Correlation Scores by Sliding the Segment")
        axs[1].legend()
        axs[1].set_xlim(0, number_of_track_points)
        # axs[1].set_ylim(-1, 1)

        points = track.path
        axs[2].set_title("Track Points")
        axs[2].scatter(points[:, 0], points[:, 1], color='orange', label='Original Points', s =0.5)
        l1 = track.path[:, :2] + track.nvecs * track.widths[:, 0][:, None]
        l2 = track.path[:, :2] - track.nvecs * track.widths[:, 1][:, None]
        axs[2].plot(l1[:, 0], l1[:, 1], color='black')
        axs[2].plot(l2[:, 0], l2[:, 1], color='black')
        for i, peak in enumerate(peaks):
            axs[2].scatter(points[peak, 0], points[peak, 1], color='red', marker='*', s=100)
        axs[2].scatter(points[estimated_position_peak, 0], points[estimated_position_peak, 1], color='green', label='Estimated Progress point', marker='*', s=100)
        axs[2].scatter(points[n, 0], points[n, 1], color='blue', label='Progress point', marker='*', s=100)
        plt.legend()
        plt.tight_layout()
        plt.show()

# Create an interactive slider for `n` with a longer width
slider = IntSlider(min=0, max=1007, step=1, value=300, layout=Layout(width='800px'))
interact(plot_with_slider, n=slider);


interactive(children=(IntSlider(value=300, description='n', layout=Layout(width='800px'), max=1007), Output())…